In [6]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import itertools

import torch
from torch import nn

import youtokentome as yttm

from utils.poetry import *
from utils.transformer_tools import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def gen_some_sent(gen_model, phrase, N, beam_size, need_reweight=False, temperature=0, alpha=0):
    """
    Функция для генерации нескольких различных продолжений фразы с возможностью настройки параметров температуры и сглаживания меток
    """
    answers = []
    for i in range(N):
        beam_gen_variants = gen_model(phrase, beamsize=beam_size, return_hypotheses_n=1, need_reweight=need_reweight, temperature=temperature, alpha=alpha)
        for score, pred_txt in beam_gen_variants:
            answers.append(pred_txt)
    return answers

def test_gen(gen_model, text, temperatures, alphas):
    """
    Фукнция для перебора комбинаций температуры и сглаживания для генерации продолжения фразы
    """
    all_test = itertools.product(temperatures, alphas)
    for t, a in all_test:
        print(f"Температура = {t}, альфа = {a}")
        answers = gen_some_sent(gen_model, text, 5, 10, True, t, a)
        for ans in answers:
            print(ans)
        print("-----")

In [8]:
BPE_MODEL_FILENAME = './models/tokens/eragon_bpe_1000.yttm'
tokenizer = yttm.BPE(BPE_MODEL_FILENAME)

G_model = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 256,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 256,
            nhead = 16,
            dim_feedforward = 512,
            dropout = 0.1
        ),
        num_layers=5
    ),
    emb_dropout = 0.1
)

In [9]:
temps = [0.01, 0.2, 0.5, 0.8]
alps = [0, 0.1, 0.3, 0.5]

### Модель после генеративно-состязательной сети

In [10]:
G_model.load_state_dict(torch.load('./models/GAN/Eragon/model_G.pth'))
beam_generator = BeamGenerator(G_model, tokenizer)

In [11]:
test_gen(beam_generator, "Today we are", temps, alps)

Температура = 0.01, альфа = 0
Today we are captured by the Empire. We aren’t going to find the Ra’zac. They’re going to get here.” Eragon shrugged. “I’m sorry,” said Brom
Today we are in the middle of the Beor Mountains.” Eragon shook his head. “Maybe,” said Brom, “but I can’t be able to get the
Today we are able to escape the Empire, but we can’t find the Ra’zac.” “Maybe you’re going to get there,” said Brom. “Of course
Today we are in the middle of the center of Du Weldenvarden?” “Of course,” said Orik, “but perhaps it is not the
Today we are supposed to the Varden.” “Maybe,” said Eragon, pushing his hands. “My Lady Nightstalker.” After a minute, he
-----
Температура = 0.01, альфа = 0.1
Today we are busy.” “Probably,” said Brom. “Yes,” said Eragon defensively. “They’re going to be able to find the Ra’zac. They’ll
Today we are busy.” “Yes, Master.” “Yes, Master,” said Oromis, “y Lady Nightstalker.” After a minute, Eragon said, “Th
Today we are alone, but I will be able to get out of t

In [12]:
test_gen(beam_generator, "For the King and", temps, alps)

Температура = 0.01, альфа = 0
For the King and Maudhí Blödhren, Feldûnost, For the Council of Tel’naeír’s Nolfavre
For the King and hurried back toward the center of Tronjheim’s central chamber. In the middle of Tronjheim’s central
For the King and the Crags of Tel’naeír — a bristae of the Lethrblaka and Nolfavrell, who had
For the King and Narí Baldor said, “I will be afraid, Shadeslayer, Shadeslayer, Shadeslayer, Shadeslayer.
For the King and Nolfavrell to the south of Teirm and the other dwarves. Atra esterní Blödhren, Hrothgar
-----
Температура = 0.01, альфа = 0.1
For the King and Farthen Dûr, who had told him what he was about to say, “My Lady Nightstalka,” said Oromis, “but you have to do
For the King and diamonder of the Lethrblaka and Blood-oath Celebration. Nothing else would happen to me, but I
For the King and Surda.” “And what do you mean?” asked Nasuada. “One whole Trial of the Long Knives and Nolfavrell
For the King and Nolfavrell and Lifaen, and Narí and Blödhren, and No

In [13]:
test_gen(beam_generator, "We are going to", temps, alps)

Температура = 0.01, альфа = 0
We are going together, but I have no choice now.” “Of course,” said Brom. “Once upon the center of Morzan’s Forsw
We are going to hunt for the Urgals.” “I’m not going to go to the Varden,” said Murtagh, “but we can get there to Teirm.” Murtagh shook his head
We are going to take care of the Empire.” Eragon gazed at her, then returned to the cave. “Maybe,” said Murtagh. “Of course, I’ll
We are going to talk.” Eragon shook his head. “Maybe,” said Brom. “My Longshanks,” said Brom, “but we’ll be safe
We are going to determine ourselves.” “Yes, Master.” “Then why do you know?” “No, Ma’am.” “My Lady Nightstalk
-----
Температура = 0.01, альфа = 0.1
We are going to delay, but I can’t be able to help you.” “Perhaps,” said Eragon. He looked at the bottom. “I’m sure that you
We are going to call upon the Beor Mountains.” “Of course, Shadeslayer?” “Yes, Master.” Eragon shook his head. “That’s a
We are going to Teirm.” “Yes,” said Roran, “but we’ll be safe for the sold

In [14]:
test_gen(beam_generator, "That long story is", temps, alps)

Температура = 0.01, альфа = 0
That long story is madness.” “And what happens?” asked Eragon. “For now,” said Brom. “Of course not, but I can’t be able to use magic.
That long story is since we first arrived in Tronjheim. We have to be able to escape. It’s a difference, but we’ve never seen
That long story is captured at the city of Teirm’s Ninor River, and the Twins of Dras-Leona Lake Kn
That long story is, but I don’t know. . Why doesn’t you tell me about this? Wouldn’t you tell me? Would you tell me? Why? Why didn’t you tell
That long story is months.” “Perhaps,” said Brom, “but we can’t be able to use it.” “My Lady Nightstalker.”
-----
Температура = 0.01, альфа = 0.1
That long story is beyond our family.” “And what of the Varden?” asked Eragon. “Of course.” “As you wish to know?” “Yes, Ma’am
That long story is, but it isn’t necessary.” “Aye,” said Brom. “I’m not going to be able to take myself,” said Brom. “One of
That long story is in the middle of the Empire. . .” He pulled the ta

### Модель до генеративно-состязательной сети

In [15]:
G_model.load_state_dict(torch.load('./models/Lord/Eragon/Lord.pth'))
beam_generator = BeamGenerator(G_model, tokenizer)

In [16]:
test_gen(beam_generator, "Today we are", temps, alps)

Температура = 0.01, альфа = 0
Today we are gone,” said Eragon with a smile. “Of course, I would be able to help us. . Well. . Well, if you wish to
Today we aren’t going together, but we can’t go.” “Of course,” said Murtagh, “but we’ll be able to help you.” Murtagh shook his head
Today we are in the middle of a center of the Empire.” “My Lady Nightstalker?” asked Eragon. “Yes, Ma’am, Ma’am
Today we aren’t going tomorrow.” “Aye,” said Eragon. “One of the Council of Elders.” “Then why doesn’t you tell me what
Today we are already except for our family.” “And I am a Rider?” “Of course,” said Orik, “you can’t be afford to
-----
Температура = 0.01, альфа = 0.1
Today we are in the Beor Mountains.” “Yes, Master?” “Yes, Master.” The elf’s face darkened. “Of course.” Eragon nodded and said,
Today we are done,” said Eragon, “that you can do that.” “Perhaps we have told me.” “Perhaps,” said Murtagh, “but we
Today we are supposed to the Varden.” “My Lady Nightstalker?” “Yes,” said Nasuada. “Of cour

In [17]:
test_gen(beam_generator, "For the King and", temps, alps)

Температура = 0.01, альфа = 0
For the King and pouring Palancar Valley and Nolfavrell to the southern edge of Tronjheim’s citadel. The
For the King and Black Hrothgar, and the Jiet River Agaetí Blödhren — Feldûnost — Feldûnost
For the King and relieved the village of Carvahall’s villagers had been seen in Palancar Valley. He was aware of the villagers
For the King and Narí Blödhren, and the Cripple Who Is Who Is Whole Is Whole Whole Who
For the King and Narí Baldor, and Narí and Loring Loring, Birgit, and Nolfavrell, and Narílfavrell
-----
Температура = 0.01, альфа = 0.1
For the King and Lifaen, and Narí’s Narí and Lifaen. Narí and Lifaen and Narí, and Narí and Lifa
For the King and For Ma’am, Master Gany Feldûnost, and For the Forsworn and the Council of Elders Leona
For the King and Beloth the Wise and the Crags of Tel’naeír, Blödhren, and the Crags of Tel’na
For the King and Naría Lifaen, and Lifaen and Nolfavrell, Lifaen, and Narí and Narí and Lifaen
For the King and Sloan’s true n

In [18]:
test_gen(beam_generator, "We are going to", temps, alps)

Температура = 0.01, альфа = 0
We are going to take the first time.” “Of course,” said Brom. “Yes, I don’t have any idea what you want to do. It’s a good idea to be
We are going to capture, but I will be able to escape the Empire.” “I’m not sure,” said Murtagh. “Of course,” said Murtagh, “
We are going to go to the Varden, but we’ll have to take the Varden to the Varden and the Varden. We can’t defeat Galbatorix’s soldiers until we leave Teirm. That
We are going to take care of the Varden. We’ll be able to help us,said Saphira.I’m sure we’re going to go to the Varden,said Saphira. She
We are going to delay, and we’ll be able to escape the Empire. We’ll be safe,” said Murtagh. “I’m sorry,” said Murtagh, “b
-----
Температура = 0.01, альфа = 0.1
We are going to be able to escape the Empire.” “If we’re not going to take care of the Ra’zac,” said Brom. “Yes,” said Brom, “but we
We are going to hunt. We have to do that,” said Eragon. “Of course.” He paused. “I’m not sure,” said Brom, “but we 

In [19]:
test_gen(beam_generator, "That long story is", temps, alps)

Температура = 0.01, альфа = 0
That long story is your name?” “Of course.” “Yes,” said Brom. “Of course.” “One of my family.” “First
That long story is in the midst of the Spine.” “And what do you think?” asked Eragon. “Once upon the Spine,” said Brom. “Of course
That long story is the most important thing.” “Eragon,” said Brom. “Of course, I would be able to find out what I can do.” “Very well
That long story is capable of Alagaësia.” “Aye,” agreed Roran. “Yes, Master. My Lady Nightstalker.” He p
That long story is unconscious.” Eragon shook his head. “My Lady,” said Brom. “Yes, Master. I know you’ve never seen anything else. It
-----
Температура = 0.01, альфа = 0.1
That long story is since we first arrived in Alagaësia.” “Aye. .?” asked Eragon. “Of course, I have no choice. It’s a
That long story is.” “And what is it?” asked Eragon. “Of course hesitated,” said Brom. “Of course he’s a good name. I have to
That long story is, but I have no choice.” “One of course.” “Perhaps,” said Brom,